## Fake News Corpus
### https://github.com/several27/FakeNewsCorpus/releases/tag/v1.0
Mapping: (tag)  ->  label
- (satire)      ->  1-"Satire"
- (conspiracy)  ->  2-"Hoax"
- (bias)        ->  3-"Propaganda"
- (reliable)    ->  4-"Reliable News"

In [4]:
import sys
import csv
import pandas as pd

from imblearn.under_sampling import RandomUnderSampler

In [2]:
csv.field_size_limit(500000)

131072

In [5]:
CHUNK_SIZE = 10 ** 6

In [6]:
with pd.read_csv('news_cleaned_2018_02_13.csv', chunksize=CHUNK_SIZE, index_col="id", engine='python') as reader:
    for chunk in reader:
        type_content = chunk[['type', 'content']]
        filtered = type_content[(type_content.type == 'satire') | (type_content.type == 'conspiracy') | (type_content.type == 'bias') | (type_content.type == 'reliable')]
        filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
        filtered.to_csv('fakenewscorpus.csv', mode='a', index=False, header=False)

C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A

In [ ]:
df = pd.read_csv('fakenewscorpus.csv', header=None, index_col=False)

In [5]:
X = df.iloc[:, 1]
y = df.iloc[:, 0]

print(X.head())
print(y.value_counts())

0    \n\n\n\n\n\n\n\nRev Dr. Childress is available...
1    \n\nSpeaking Engagement Request\n\n\n\nContact...
2    "…I have set before you life and death, blessi...
3    Why We Oppose Planned Parent Hood ( The follow...
4    For as long as he can remember, Malcolm Turnbu...
Name: 1, dtype: object
0
4    1913222
3    1138998
2     831235
1     112948
Name: count, dtype: int64


In [7]:
rus = RandomUnderSampler(sampling_strategy={1: 15000, 2: 15000, 3: 15000, 4: 15000})
X_undersampled, y_undersampled  = rus.fit_resample(X.to_frame(), y.to_frame())

In [8]:
print(X_undersampled.head())
print(y_undersampled.value_counts())

                                                         1
853618   Ein Ordner versucht, Klopp zu halten – vergebl...
397186   Local\n\nArea Man Afraid Some Woman Might Come...
1643627  As Spider-Man comics have always taught us, wi...
393069   Swedish researchers have found that the brains...
389510   Local\n\nArea Man Afraid Some Woman Might Come...
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [10]:
df_undersampled = pd.concat([y_undersampled, X_undersampled], axis=1)

In [12]:
X_undersampled = df_undersampled.iloc[:, 1]
y_undersampled = df_undersampled.iloc[:, 0]

print(X_undersampled.head())
print(y_undersampled.value_counts())

853618     Ein Ordner versucht, Klopp zu halten – vergebl...
397186     Local\n\nArea Man Afraid Some Woman Might Come...
1643627    As Spider-Man comics have always taught us, wi...
393069     Swedish researchers have found that the brains...
389510     Local\n\nArea Man Afraid Some Woman Might Come...
Name: 1, dtype: object
0
1    15000
2    15000
3    15000
4    15000
Name: count, dtype: int64


In [13]:
df_undersampled.to_csv('fakenewscorpus_balancedtest.csv', index=False, header=False)